# PROBLEMA DO CAMINHO MÍNIMO


O _Problema do Caminho Mínimo_ é o seguinte: dados um grafo ponderado (com pesos, ou distâncias) e dois de seus vértices, encontrar o caminho nesse grafo que ligue os dois vértices dados e tenha o menor peso (ou distância) total possível.  Uma descrição mais detalhada, incluindo definições e exemplos dos conceitos necessários para entender esse problema, está contida nas notas <a href="https://github.com/Jvfc745/IC---Problemas-de-Caminho-minimo/blob/4eebbdba79f21bc0d4b9bb949cb663c7e6c9fe53/Joao%2C%20Rodrigo%20-%20Problema%20do%20caminho%20minimo.pdf">Problema do Caminho Mínimo</a>.

Nesse _notebook_, serão apresentadas implementações comentadas de três algoritmos usados para resolver esse problema: _Busca Exaustiva_, uma _heurística gulosa_, e o _Algoritmo de Dijkstra_.


## Bibliotecas

Para começar, vamos carregar as duas bibliotecas do ``Python`` que serão usadas nesse _notebook_.  A primeira delas, ``heapq``, será usada no Algoritmo de Dijkstra para criar listas de prioridades. A segunda, ``time``, será usada nos três algoritmos para calcular os seus tempos de execução.


In [ ]:
import heapq
import time


## Implementações dos algoritmos


Essa é uma função auxiliar, que será usada para verificar se existe algum caminho entre dois vértices de um grafo.  Concretamente, ela constrói o conjunto de todos os vértices que estão conectados ao vértice-base, ou seja, a <a href="http://en.wikipedia.org/wiki/Connectivity_(graph_theory)#Components_and_cuts">componente conexa</a> do vértice-base no grafo.


In [ ]:
def componente_conexa (grafo, vertice_base):
    if vertice_base in grafo:
        componente_conexa = set()
        vizinhos = {vertice_base}

        while componente_conexa != vizinhos:
            componente_conexa = vizinhos.copy()
            for vertice in componente_conexa:
                vizinhos.update(grafo[vertice])

        return componente_conexa

    else:
        print(f"O vértice {vertice_base} não pertence ao grafo.")
        return None


### **Busca Exaustiva**

O primeiro algoritmo para calcular o caminho mínimo entre dois vértices de um grafo consiste em construir todos os caminhos possíveis entre um _vértice inicial_ e um _vértice final_, e depois escolher o caminho que tem menor peso (distância).  Como esse algoritmo busca o caminho mínimo dentre todos os possíveis caminhos, ele é chamado de _Busca Exaustiva_.


In [ ]:
def busca_exaustiva (grafo, vertice_inicial, vertice_final):
    """
    Busca exaustiva: dado um grafo ponderado e dois de seus vértices, realiza
    uma busca exaustiva para encontrar o caminho mínimo entre esses vértices.

    Parâmetros:
    - grafo (dict): Um dicionário representando o grafo, onde as chaves são os
      vértices e os valores são listas de vértices vizinhos e suas distâncias.
    - vertice_inicial: O vértice de partida para a busca.
    - vertice_final: O vértice de destino para a busca.

    Retorna: uma tupla contendo:
    - caminho_minimo (list): uma lista representando o caminho mínimo entre o
       vértice inicial e o vértice final.
    - menor_distância (float): um número representando o comprimento total do
      caminho mínimo.
    - tempo_execucao (float): um número representando o tempo total de
      execução do algoritmo.

    Complexidade: O(d^|V|), d é o grau do grafo e |V| é o número de vértices
      do grafo dado.
    """

    # Validações básicas
    if not componente_conexa(grafo, vertice_inicial):
        return None, None, None
    elif vertice_final not in grafo:
        print(f"O vértice final {vertice_final} não pertence ao grafo.")
        return None, None, None
    elif vertice_final not in componente_conexa(grafo, vertice_inicial):
        return None, None, None
    #else: como os dois vértices pertencem ao grafo dado e como existe um
    #  caminho entre o vértice inicial e o vértice final, continuamos...

    # Horário do início do algoritmo
    horario_inicio = time.time()

    # Lista que vai guardar os caminhos já construídos e seus comprimentos.
    lista = []

    # Lista que vai guardar os caminhos expandidos. Inicializamos essa lista
    # só com o vértice inicial, que é como se fosse uma expansão da lista
    # vazia.
    nova_lista = [{'caminho': [vertice_inicial], 'comprimento': 0}]

    # Enquanto os caminhos ainda puderem ser expandidos:
    while nova_lista != lista:
        # Atualizamos a lista de caminhos já construídos
        lista = nova_lista

        # Construímos uma nova lista de caminhos expandidos
        nova_lista = []

        for elemento in lista:
            novos_elementos = expandir_caminho(grafo, vertice_final, elemento['caminho'], elemento['comprimento'])
            nova_lista += [{'caminho': caminho, 'comprimento': comprimento}  for caminho, comprimento in novos_elementos]

    # Quando os caminhos não puderem mais ser expandidos, calcular o caminho
    # mínimo e seu comprimento
    menor_distancia = min([item['comprimento']  for item in lista])
    caminho_minimo = [item['caminho']  for item in lista if item['comprimento'] == menor_distancia][0]

    # Calcular o tempo tota de execução do algoritmo
    tempo_execucao = time.time() - horario_inicio

    return caminho_minimo, menor_distancia, tempo_execucao


def expandir_caminho(grafo, vertice_final, caminho, comprimento):
    # Tomar o último vértice do caminho
    ultimo_vertice = caminho[-1]

    # Se o último vértice já for o vértice final, não precisa expandir
    if ultimo_vertice == vertice_final:
        return [(caminho, comprimento)]

    # Se não, pegamos todos os vizinhos do último vértice que ainda não estão
    # no caminho e expandimos o caminho inicial com cada um deles
    else:
        vizinhos_nao_visitados = {(vertice, distancia)  for vertice, distancia in grafo[ultimo_vertice].items() if vertice not in caminho}
        caminhos_expandidos = [([*caminho, vertice], comprimento + distancia)  for vertice, distancia in vizinhos_nao_visitados]
        return caminhos_expandidos


### **Heurística Gulosa**

Observe que, como a Busca Exaustiva compara todos os caminhos entre os vértices inicial e final, seu custo computacional acaba crescendo rápido conforme o tamanho do grafo cresce.  Para atenuar isso, o próximo algoritmo abre mão de encontrar o caminho mínimo para ter um custo computacional menor.  Por isso, esse método é chamado de _heurístico_.


In [ ]:
def heuristica_gulosa (grafo, vertice_inicial, vertice_final):
    """
    Heurística Gulosa: dados um grafo ponderado e dois vértices, essa função
    encontra um caminho do vértice inicial até o vértice final que segue as
    arestas de peso mínimo. (Esse caminho pode não ser o caminho mínimo!)

    Parâmetros:
    - grafo (dict): Dicionário de adjacências que representa o grafo
      ponderado. As chaves são os vértices do grafo e os valores são
      dicionários, cujas chaves são os vizinhos e os valores são os pesos das
      arestas que ligam os vértices.
    - vertice_inicial: Vértice inicial do caminho. Se esse vértice não
      pertencer ao grafo, a função retorna None (e uma mensagem de erro).
    - vertice_final: Vértice final do caminho. Se esse vértice não pertencer
      ao grafo, a função retorna None (e uma mensagem de erro).

    Saídas:
    - caminho (list): Uma lista contendo os vértices do caminho encontrado.
    - peso_total (float): Soma dos pesos das arestas do caminho encontrado.
    - tempo_execucao (float): Tempo total de execução da heurística (em segs).

    Complexidade: O(|V|^2 + |E|), onde |V| é o número de vértices e |E| é o
      número de arestas do grafo dado.
    """

    # Começo do cálculo do tempo de execução do algoritmo
    horario_inicio = time.time()

    # Validações básicas
    if not componente_conexa(grafo, vertice_inicial):
        return None, None, None
    elif vertice_final not in grafo:
        print(f"O vértice final {vertice_final} não pertence ao grafo.")
        return None, None, None
    elif vertice_final not in componente_conexa(grafo, vertice_inicial):
        return None, None, None
    #else: como os dois vértices pertencem ao grafo dado e como existe um
    #  caminho entre o vértice inicial e o vértice final, continuamos...

    # Preparação para o loop principal:
    # - conjunto de vértices não visitados
    nao_visitados = set(grafo)
    # - lista que vai guardar o caminho do vértice inicial ao final
    caminho = []
    # - variável que vai guardar o peso total do caminho acima
    peso_total = 0.0
    # - vértice que estará sendo analisado em cada iteração do loop
    vertice_atual = vertice_inicial

    # Loop que só para quando chegamos ao vértice final, ou ...
    while vertice_atual != vertice_final:
        # Incluir o vértice atual ao caminho
        caminho.append(vertice_atual)
        # Remover o vértice atual do conjunto de vértices não visitados
        nao_visitados.discard(vertice_atual)
        # Dicionário contendo os vizinhos do vértice atual que ainda não foram
        # visitados e os pesos das respectivas arestas
        vizinhos_nao_visitados = {vizinho: peso  for vizinho, peso in grafo[vertice_atual].items() if vizinho in nao_visitados}

        if vizinhos_nao_visitados:
            # O vértice atual para a próxima iteração será o vizinho mais próximo
            vertice_atual = min(vizinhos_nao_visitados, key=vizinhos_nao_visitados.get)
            # Adicionar o tamanho da aresta ligando os vértices atuais ao peso total
            peso_total += vizinhos_nao_visitados[vertice_atual]
        else:
            #TODO  Sugestão: ao invés de 'break', nós poderíamos dar um passo
            #       atrás (voltar um vértice) e continuar tentando...
            break

    if vertice_atual == vertice_final:
        # Incluir o último vértice no caminho
        caminho.append(vertice_atual)

        # Calcular o tempo total de execução do algoritmo
        tempo_execucao = time.time() - horario_inicio

        return caminho, peso_total, tempo_execucao

    else: # Não foi possível alcançar o vértice final com a heurística gulosa
        return None, None, None


### **Algoritmo de Dijkstra**

O terceiro algoritmo, o _Algoritmo de Dijkstra_, junta os pontos positivos dos dois algoritmos acima.  Ele usa a _Heurística Gulosa_ para encontrar o caminho mínimo sem necessariamente construir todos os caminhos.  Dessa forma, ele sempre retorna o caminho mais curto entre o vértice inicial e todos os outros vértices do grafo e, em geral, de maneira mais rápida que a _Busca Exaustiva_.


In [ ]:
def dijkstra (grafo, vertice_inicial):
    """
    Algoritmo de Dijkstra: dado um grafo ponderado e um vértice inicial,
    encontra os caminhos mínimos desse vértice inicial até os outros vértices
    do grafo.

    Parâmetros:
    - grafo (dict): Dicionário de adjacências que representa o grafo
      ponderado. As chaves são os vértices do grafo e os valores são
      dicionários, cujas chaves são os vizinhos e os valore são os pesos das
      arestas que ligam os vértices.

    - vertice_inicial (dict_keys): O vértice inicial dos caminhos mínimo que
      queremos calcular. Se o vértice dado não pertencer ao grafo dado, a
      função retorna None (e imprime uma mensagem de erro).

    Saídas:
    - distancias (dict): Um dicionário contendo os caminhos mínimos do vértice
      inicial aos outros vértices do grafo. As chaves desse dicionário são os
      vértices do grafo e os valores são listas formadas pela distância total
      e pelo último vértice do caminho mínimo.

    - tempo_execucao (float): Tempo total de execução do algoritmo (em segs).

    Complexidade: O(|E| + |V|*log|V|), onde |E| é o número de arestas e |V| é
      o número de vértices do grafo dado.
    """

    # Começo do cálculo do tempo de execução do algoritmo (O(|E| + |V|log|V|))
    horario_inicio = time.time()

    # Validação básica
    if vertice_inicial not in grafo:
        print(f"O vértice inicial {vertice_inicial} não pertence ao grafo.")
        return None, None
    #else: como o vértice inicial pertence ao grafo dado, continuamos...

    # Dicionário criado para alocar: a menor distância entre o vértice inicial
    # e os outros vértices do grafo, e o caminho mínimo. Esse dicionário é
    # inicializado com infinito para todos os vértices (porque ainda não
    # calculamos nada).
    distancias = {vertice: [float('infinity'), vertice_inicial]  for vertice in grafo}

    # Depois definimos a distância do vértice inicial a ele mesmo como sendo 0
    distancias[vertice_inicial][0] = 0

    # Definição do conjunto de vértices que ainda precisam ser analisados
    nao_visitados = {vertice  for vertice in grafo}

    # Definição de uma lista que, junto com o conjunto de vértices não-
    # -visitados, vai ser usada para determinar o próximo vértice a ser
    # analisado. Essa lista é inicializada com (0, vertice_inicial) por dois
    # motivos: para ser não-vazia, e para que a primeira iteração do loop
    # inclua nela todos os vizinhos do vértice inicial.
    fila_prioridade = [(0, vertice_inicial)]

    # Observe que a fila de prioridade vai ser vazia quando não existir mais
    # nenhum vizinho a ser analisado
    while fila_prioridade:
        # Reordenar a fila de prioridade por ordem de distância e extrair o
        # primeiro item dessa lista. Esse vértice será o vizinho mais próximo
        # do vértice anterior que ainda não foi analisado.
        heapq.heapify(fila_prioridade)
        distancia_atual, vertice_atual = heapq.heappop(fila_prioridade)

        # Se a distância atual é maior do que a distância armazenada, ignore
        if distancia_atual > distancias[vertice_atual][0]:
            continue
        else:
            for vizinho, peso in grafo[vertice_atual].items():
                # Para cada vizinho do vértice atual, verificar se a distância
                # do caminho que passa pelo vértice atual e vai para esse
                # vizinho é menor que a distância que nós tínhamos calculado
                # antes. Se for, atualiza o valor da distância mínima.
                if distancia_atual + peso < distancias[vizinho][0]:
                    distancias[vizinho] = [distancia_atual + peso, vertice_atual]
                #else: mantemos a distância que nós já tínhamos, que é menor

            # Como o vértice atual já foi analisado, exclua ele do conjunto de
            # vértices não-visitados
            nao_visitados.discard(vertice_atual)

            # Atualize a fila de prioridade com os vizinhos não-visitados do
            # vértice atual
            fila_prioridade = [(distancias[vizinho][0], vizinho)  for vizinho in list(grafo[vertice_atual]) if vizinho in nao_visitados]

    # Cálculo do tempo total de execução
    tempo_execucao = time.time() - horario_inicio

    return reconstruir_caminhos(distancias, vertice_inicial), tempo_execucao


def reconstruir_caminhos (dicionario_distancias, vertice_inicial):
    caminhos_minimos = {}

    for a in dicionario_distancias:
        ultimo = a;
        caminho = [a]

        while ultimo != vertice_inicial:
            caminho.insert(0, dicionario_distancias[ultimo][1])
            ultimo = dicionario_distancias[ultimo][1]

        caminhos_minimos[a] = [dicionario_distancias[a][0], caminho]

    return caminhos_minimos


## Exemplos

Para terminar, vamos fazer alguns exemplos de uso dos algoritmos construídos acima.


__Exemplo 1.__
Considere o seguinte grafo

<p align="center">
  <img src="http://github.com/Jvfc745/IC---Problemas-de-Caminho-minimo/blob/9f9cacf9279856b97ac3b4e233e89ff47a0d79f3/imagens/grafo1.png?raw=true" />
</p>

Observe que, nesse caso, o caminho mínimo entre os vértices $a$ e $d$ tem comprimento 1.  Vamos rodar os três algoritmos acima para comparar seus resultados.


Para começar, nós precisamos escrever a [_lista de adjacências_](https://en.wikipedia.org/wiki/Adjacency_list) do grafo acima.  Na implementação dos algoritmos que nós fizemos, é mais conveniente representar essa lista de adjacências como um _dicionário_ de `Python`:


In [ ]:
grafo = {'a': {'b': 1, 'c': 1, 'd': 1},
         'b': {'a': 1, 'c': 1, 'd': 1},
         'c': {'a': 1, 'b': 1, 'd': 1},
         'd': {'a': 1, 'b': 1, 'c': 1}}


Agora, usando essa representação do grafo como uma lista de adjacências, nós podemos aplicar os três algoritmos construídos na seção anterior:


In [ ]:
print("Busca Exaustiva:", busca_exaustiva(grafo, 'a', 'd'))

print("Heurística Gulosa:", heuristica_gulosa(grafo, 'a', 'd'))

print("Algoritmo de Dijkstra:", dijkstra(grafo, 'a'))


Busca Exaustiva: (['a', 'd'], 1, 5.4836273193359375e-05)
Heurística Gulosa: (['a', 'b', 'c', 'd'], 3.0, 3.409385681152344e-05)
Algoritmo de Dijkstra: ({'a': [0, ['a']], 'b': [1, ['a', 'b']], 'c': [1, ['a', 'c']], 'd': [1, ['a', 'd']]}, 3.0994415283203125e-05)


Observe que as 3 entradas da função `busca_exaustiva` são: a lista de adjacências do grafo (`grafo`), o ponto inicial ($a$), e o ponto final do caminho ($d$).  Enquanto as suas 3 saídas são: o caminho mínimo ($a \to d$), o peso total desse caminho ($1$), e o seu tempo de execução (~$5,5 \cdot 10^{-5}$ segundos).

De forma similar, as 3 entradas da função `heuristica_gulosa` são: a lista de adjacências de um grafo, o ponto inicial e o ponto final do caminho.  Enquanto as suas 3 saídas são: o caminho encontrado ($a \to b \to c \to d$), o peso total desse caminho ($3$), e o seu tempo de execução (~$3,4 \cdot 10^{-5}$ segundos).  Observe que, nesse caso, a Heurística Gulosa _não_ retorna o caminho mínimo.

Já a função `dijkstra` tem só 2 entradas: a lista de adjacências de um grafo e o ponto inicial do caminho.  As suas saídas contêm: um dicionário e o seu tempo de execução (~$3,1 \cdot 10^{-5}$ segundos).  Nesse dicionário então os pontos finais, o peso total do caminho mínimo do ponto inicial até cada um desses pontos finais, e os respectivos caminhos mínimos.  Assim, por exemplo, o caminho mínimo de $a$ até $d$ tem peso total $1$ e o caminho é $a \to d$.

Observe também que o Algoritmo de Dijkstra é mais rápido que a Heurística Gulosa, que por sua vez, é mais rápida que a Busca Exaustiva.


__Exemplo 2.__
Para esse exemplo, considere o grafo
<p align="center">
  <img src="https://github.com/Jvfc745/IC---Problemas-de-Caminho-minimo/blob/1d2cfea4a119d72ad35ede4d1820daf6e20385c3/imagens/grafo2.png?raw=true" />
</p>

Observe que sua lista de adjacências é a seguinte:


In [ ]:
grafo = {'a': {'b': 1, 'c': 2},
         'b': {'a': 1, 'd': 3},
         'c': {'a': 2, 'd': 4},
         'd': {'b': 3, 'c': 4}}


Vamos aplicar os três algoritmos acima e analizar seus resultados:


In [ ]:
print("Busca Exaustiva:", busca_exaustiva(grafo, 'a', 'd'))

print("Heurística Gulosa:", heuristica_gulosa(grafo, 'a', 'd'))

print("Algoritmo de Dijkstra:", dijkstra(grafo, 'a'))


Busca Exaustiva: (['a', 'b', 'd'], 4, 2.3365020751953125e-05)
Heurística Gulosa: (['a', 'b', 'd'], 4.0, 2.0265579223632812e-05)
Algoritmo de Dijkstra: ({'a': [0, ['a']], 'b': [1, ['a', 'b']], 'c': [2, ['a', 'c']], 'd': [4, ['a', 'b', 'd']]}, 2.09808349609375e-05)


Observe que as 3 entradas da função `busca_exaustiva` são: a lista de adjacências do grafo (`grafo`), o ponto inicial ($a$) e o ponto final ($d$) do caminho.  Enquanto as suas saídas são: o caminho mínimo ($a \to b \to d$), o peso total desse caminho ($4$), e o seu tempo de execução (~$2,34 \cdot 10^{-5}$ segundos).

De forma similar, as 3 entradas da função `heuristica_gulosa` são: a lista de adjacências do grafo, o ponto inicial e o ponto final do caminho.  Enquanto as suas saídas são: o caminho encontrado ($a \to b \to d$), o peso total desse caminho ($4$), e o tempo de execução (~$2,03 \cdot 10^{-5}$ segundos).  Observe que, nesse caso, a Heurística Gulosa retorna exatamente o caminho mínimo, de forma mais rápida que a Busca Exaustiva.

Já a função `dijkstra` tem só 2 entradas: a lista de adjacências do grafo e o ponto inicial.  As suas saídas contêm: um dicionário e o seu tempo de execução (~$2,1 \cdot 10^{-5}$ segundos).  Nesse dicionário então os pontos finais, o peso total do caminho mínimo do ponto inicial até cada um desses pontos finais, e os respectivos caminhos mínimos.  Assim, por exemplo, o caminho mínimo de $a$ até $d$ tem peso total $4$ e o caminho é $a \to b \to d$.

Observe também que, nesse caso, a Heurística Gulosa é mais rápida que o Algoritmo de Dijkstra, que por sua vez, é mais rápido que a Busca Exaustiva.  Por fim, observe que, nesse caso, os três algoritmos retornam o caminho mínimo antre $a$ e $d$.
